# AgentCore Gateway での MCP サーバーのツール定義の更新

## 概要

AgentCore Gateway の同期メカニズムは、MCP サーバーターゲットから正確なツール定義を保証します。明示的および暗黙的な同期方法を通じて、スキーマの一貫性を管理し、パフォーマンスを最適化し、データの整合性を維持します。

### 明示的同期

SynchronizeGateway API は、MCP サーバーターゲットからツールのオンデマンド同期を可能にする非同期操作です。この API は、ツール定義をいつリフレッシュするかを顧客が明示的に制御できるようにし、MCP サーバーのツール設定を変更した後に特に有用です。OAuth 認証で設定されたターゲットの場合、API は MCP サーバー通信を進める前に AgentCore Identity サービスを通じて認証情報を検証します。検証が失敗した場合、同期操作は適切なエラー詳細で失敗し、ターゲットを FAILED 状態に遷移させます。認証なしで設定されたターゲットの場合、API は直接ツール同期に進みます。

ツール処理ワークフローは、MCP サーバーとのセッションの確立から始まります。その後、API は最適化されたバッチでツールを取得して処理し、他のターゲットのツールとの命名衝突を防ぐためにターゲット固有のプレフィックスを追加します。API はターゲットあたり 10,000 ツールの制限を強制し、元の MCP サーバー定義からの重要なメタデータを保持しながらツール定義が正規化されることを保証します。

このチュートリアルでは、'01-Gateway-MCP-Target' で作成した MCP サーバーに追加ツールを追加して更新し、SynchronizeGateway API を呼び出して最新のツール定義を明示的に同期します。

![Diagram](images/mcp-server-target-explicit-sync.png)

### 暗黙的同期

AgentCore Gateway は、明示的な SynchronizeGateway API とは異なる CreateGatewayTarget および UpdateGatewayTarget 操作中に暗黙的な同期を実行します。この組み込みの同期により、新しく作成または更新された MCP ターゲットは、データの一貫性を維持しながら即座に使用可能になります。この暗黙的な同期により、MCP ターゲットは常に有効で最新のツール定義で作成または更新され、READY 状態のすべてのターゲットが即座に使用可能であるという Gateway の保証が維持されます。

このチュートリアルでは、新しい MCP サーバーを作成し、UpdateGatewayTarget 操作を呼び出して、新しい MCP サーバーからツール定義を暗黙的に同期する新しい MCP ターゲットを更新します。

![Diagram](images/mcp-server-target-implicit-sync.png)

### チュートリアル詳細


| 情報                 | 詳細                                                                  |
|:---------------------|:---------------------------------------------------------------------|
| チュートリアルタイプ  | インタラクティブ                                                       |
| AgentCore コンポーネント | AgentCore Gateway、AgentCore Identity、AgentCore Runtime           |
| エージェントフレームワーク | Strands Agents                                                    |
| Gateway ターゲットタイプ | MCP サーバー                                                         |
| エージェント         | Strands                                                               |
| インバウンド認証 IdP  | Amazon Cognito（他も使用可能）                                         |
| アウトバウンド認証    | Amazon Cognito（他も使用可能）                                         |
| LLM モデル           | Anthropic Claude Sonnet 4                                             |
| チュートリアル構成    | MCP ターゲット付き AgentCore Gateway の作成とツールの同期               |
| チュートリアル分野    | クロスバーティカル                                                     |
| 例の複雑さ           | 簡単                                                                   |
| 使用 SDK             | boto3                                                                 |

## チュートリアルアーキテクチャ

このチュートリアルでは、AgentCore Gateway 内の MCP サーバーターゲットからのツール定義の明示的および暗黙的な同期をトリガーする方法について説明します。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
# Install from the requirements file or pyproject.toml file in current directory
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
# Import utils
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

# Setup logging 
import logging

# Configure logging for notebook environment
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# Set specific logger levels
logging.getLogger("gateway").setLevel(logging.INFO)

# 明示的同期の実行

## Gateway へのインバウンド認可用の Cognito ユーザープール情報の取得

In [ ]:
# Creating Cognito User Pool 
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "invoke",  # Just 'invoke', will be formatted as resource_server_id/invoke
    "ScopeDescription": "Scope for invoking the agentcore gateway"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
scopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
gw_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {gw_user_pool_id}")

utils.get_or_create_resource_server(cognito, gw_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

gw_client_id, gw_client_secret = utils.get_or_create_m2m_client(cognito, gw_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"Client ID: {gw_client_id}")

# Get discovery URL
gw_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/openid-configuration'
print(gw_cognito_discovery_url)

## Runtime へのインバウンド認可用の Cognito ユーザープール情報の取得

In [ ]:
# Creating Cognito User Pool
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-runtime-pool"
RESOURCE_SERVER_ID = "sample-agentcore-runtime-id"
RESOURCE_SERVER_NAME = "sample-agentcore-runtime-name"
CLIENT_NAME = "sample-agentcore-runtime-client"
SCOPES = [
    {"ScopeName": "invoke",  # Just 'invoke', will be formatted as resource_server_id/invoke
    "ScopeDescription": "Scope for invoking the agentcore gateway"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
runtimeScopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
runtime_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {runtime_user_pool_id}")

utils.get_or_create_resource_server(cognito, runtime_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

runtime_client_id, runtime_client_secret = utils.get_or_create_m2m_client(cognito, runtime_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"Client ID: {runtime_client_id}")

# Get discovery URL
runtime_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{runtime_user_pool_id}/.well-known/openid-configuration'
print(runtime_cognito_discovery_url)

## 既存の MCP サーバーに新しいツールを追加

'01-mcp-server-target' チュートリアルで作成した既存の MCP サーバーに新しいツール 'cancelOrder' を追加します。

In [ ]:
%%writefile mcp_server_updated.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def getOrder() -> int:
    """Get an order"""
    return 123

@mcp.tool()
def updateOrder(orderId: int) -> int:
    """Update existing order"""
    return 456

@mcp.tool()
def cancelOrder(orderId: int) -> int:
    """cancel existing order"""
    return 789

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

次に、新しい 'cancelOrder' ツールを含む更新された MCP サーバーをデプロイするために AgentCore Runtime を設定します。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server_updated.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")
agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            runtime_client_id
        ],
        "discoveryUrl": runtime_cognito_discovery_url
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server_updated.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="ac_gateway_mcp_server"
)
print("Configuration completed ✓")

次に、更新された MCP サーバーを AgentCore Runtime に再デプロイします。

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()

agent_arn = launch_result.agent_arn
agent_id = launch_result.agent_id

encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')

agent_url = f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'
print("Launch completed ✓")
print(f"Agent ARN: {agent_arn}")
print(f"Agent ID: {agent_id}")

## SynchronizeGateway API の呼び出し

### アウトバウンド認証プロバイダーの取得

前のノートブックで作成した既存の OAuth2 クレデンシャルプロバイダーを取得します

In [ ]:
identity_client = boto3.client('bedrock-agentcore-control', region_name=region)

# Get credential provider 
response = identity_client.get_oauth2_credential_provider(name="ac-gateway-mcp-server-identity")
cognito_provider_arn = response['credentialProviderArn']
print(cognito_provider_arn)

### 現在のツール定義の確認

Bedrock AgentCore Gateway を使用して MCP ツールをリストする Strands エージェントを呼び出しましょう。Gateway に同期を依頼していないため、古いツールリスト（'cancelOrder' なし）が返されます。

前のノートブックで作成した Gateway 識別子を取得します。

In [ ]:
gateway_name = 'ac-gateway-mcp-server'

gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
list_gateways_response = gateway_client.list_gateways()

gatewayID = next(
    g['gatewayId'] for g in list_gateways_response['items'] 
    if g['name'] == gateway_name
)

print(gatewayID)

Gateway URL を取得します。

In [ ]:
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
gateway_response = gateway_client.get_gateway(gatewayIdentifier=gatewayID)

gatewayURL = gateway_response['gatewayUrl']

print(gatewayURL)

In [ ]:
import json

import requests
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent


def get_token():
    token = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
    return token['access_token']


def create_streamable_http_transport():
    return streamablehttp_client(
        gatewayURL, headers={"Authorization": f"Bearer {get_token()}"}
    )


client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0", # may need to update model_id depending on region
    temperature=0.7,
    max_tokens=500,  # Limit response length
)

with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## you can replace with any model you like
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi, can you list all tools available to you")  

### SynchronizeGateway API の呼び出し

MCP ターゲット ID の取得

In [ ]:
gateway_target_name = 'mcp-server-target'

gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
list_targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gatewayID)

gatewayTargetID = next(
    g['targetId'] for g in list_targets_response['items'] 
    if g['name'] == gateway_target_name
)

print(gatewayTargetID)

明示的な同期を実行するために SynchronizeGateway API を呼び出します

In [ ]:
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

synchronize_gateway_response = gateway_client.synchronize_gateway_targets(
    gatewayIdentifier=gatewayID,
    targetIdList=[gatewayTargetID]
    )

print(synchronize_gateway_response)

### 同期後の現在のツール定義の確認

Bedrock AgentCore Gateway を使用して MCP ツールをリストする Strands エージェントを呼び出しましょう。これにより新しいツールリストが返されます。

In [ ]:
with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## you can replace with any model you like
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi, can you list all tools available to you")   

# Perform Implicit Synchronization

## 既存の MCP サーバーに別の新しいツールを追加

既存の MCP サーバーに新しいツール 'deleteOrder' を追加します。

In [ ]:
%%writefile mcp_server_updated.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def getOrder() -> int:
    """Get an order"""
    return 123

@mcp.tool()
def updateOrder(orderId: int) -> int:
    """Update existing order"""
    return 456

@mcp.tool()
def cancelOrder(orderId: int) -> int:
    """cancel existing order"""
    return 789

@mcp.tool()
def deleteOrder(orderId: int) -> int:
    """delete existing order"""
    return 101
    
if __name__ == "__main__":
    mcp.run(transport="streamable-http")

次に、新しい 'deleteOrder' ツールを含む更新された MCP サーバーをデプロイするために AgentCore Runtime を設定します。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server_updated.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")
agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            runtime_client_id
        ],
        "discoveryUrl": runtime_cognito_discovery_url
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server_updated.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="ac_gateway_mcp_server"
)
print("Configuration completed ✓")

次に、更新された MCP サーバーを AgentCore Runtime に再デプロイします。

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()

agent_arn = launch_result.agent_arn
agent_id = launch_result.agent_id

encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')

agent_url = f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'
print("Launch completed ✓")
print(f"New Agent ARN: {agent_arn}")
print(f"New Agent ID: {agent_id}")

## MCP サーバーで既存の Gateway ターゲットを更新

この Gateway ターゲット更新操作は、MCP サーバー（新バージョン）で Gateway ターゲットを更新し、暗黙的な同期を実行してツール定義を同期します

In [ ]:
import boto3

gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
update_gateway_target_response = gateway_client.update_gateway_target(
    gatewayIdentifier=gatewayID,
    targetId = gatewayTargetID,
    name = 'mcp-server-target',
    targetConfiguration={
        'mcp': {
            'mcpServer': {
                'endpoint': agent_url
            }
        }
    },
    credentialProviderConfigurations=[
        {
            'credentialProviderType': 'OAUTH',
            'credentialProvider': {
                'oauthCredentialProvider': {
                    'providerArn': cognito_provider_arn,
                    'scopes': [
                        runtimeScopeString
                    ]
                }
            }
        },
    ]
)

print(update_gateway_target_response)

## MCP サーバーターゲット更新後の現在のツール定義の確認

In [ ]:
with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## you can replace with any model you like
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi, can you list all tools available to you")   

# クリーンアップ
IAM ロール、IAM ポリシー、クレデンシャルプロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなど、追加のリソースも作成されます。クリーンアップの一環としてこれらを手動で削除する必要がある場合があります。これは実行する例によって異なります。

## Gateway の削除（オプション）

In [ ]:
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

utils.delete_gateway(gateway_client, gatewayID)

### Identity Provider の削除（オプション）

In [ ]:
identity_client.delete_oauth2_credential_provider(name='ac-gateway-mcp-server-identity')


#### Runtime の削除（オプション）

In [ ]:
import boto3
runtime_client = boto3.client('bedrock-agentcore-control',
                              region_name=REGION)

runtime_client.delete_agent_runtime(agentRuntimeId=agent_id)